In [2]:
import numpy as np
import skimage as ski
from skimage import io, transform
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] =(12,9)
import os
import copy
from IPython.display import clear_output

import torch
from torchvision import transforms
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
from torch.utils.data import DataLoader
from torch.autograd import Variable
import PIL
import torch.optim as optim

 This notebook defines an iterator though the dataset and attempts to train alexnet to take images and map them to predictions of concatenated compressed representations 

# dataset iterator

In [3]:
class Dataset:
    def __init__(self, path):
        self.path=path

    def __getitem__(self, index):
        img = PIL.Image.open(self.path+'/im-%04d.jpg'%index)
        vect = np.load(self.path+'/compressed-points-%04d.npy'%index)
        transform = transforms.Compose([transforms.ToTensor()])
        img = transform(img) 
        vect = torch.FloatTensor(np.concatenate(vect)) 
        return img, vect 

    def __len__(self):
        return len([f for f in os.listdir(self.path) if f.endswith('.jpg')])

In [4]:
train_data = Dataset('/home/kvnp/Desktop/PhD/random_projections/training/')

In [ ]:
# visually check that points info is properly encoded in the training set 
import compression as comp
import matplotlib.pyplot as plt
%matplotlib inline 
S = np.load('./sensing_matrix.npy')
for n in range(500,530):
    im = train_data[n][0][0,:,:].numpy()
    Y = train_data[n][1].numpy().reshape(5,100)
    F = comp.decode(Y,S)
    P = comp.unproject(F,im.shape,5,S.shape[1])
    comp.view_points(im,P)
    plt.show()

 # neural net

In [5]:
class AlexNet(nn.Module):

    def __init__(self,D_out=10000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, D_out),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


 # training loop

Why doesn't it train? Every parameter in model.parameters() has required_grad attribute True 

apparently only hte bias in the last linear layer has nonzero gradients, except for a few choice values



In [6]:
gpu=False 

In [23]:
model = AlexNet(D_out=500)
if gpu: model.cuda()

In [25]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [26]:
from IPython.display import clear_output

In [ ]:
k=30 #size of batch
N = 100 #number epochs
b = int(len(train_data)/k) #number of batches
losses = [] #track the losses 

running_loss = 0 #avg loss per epoch 
train_loader = DataLoader(train_data, batch_size=k, shuffle=True) #batch data loader

for epoch in range(N): 
    for i,(inputs,targets) in enumerate(train_loader): 
        #print(model.features[0]._parameters['weight'][0].data.numpy())
        clear_output(wait=True)
        #batch
        inputs,targets = Variable(inputs), Variable(targets,requires_grad=False)
        if gpu: inputs, targets = inputs.cuda(), targets.cuda() 
        #print(list(model.parameters().grad))
        #zeroed gradients 
        optimizer.zero_grad()
        
        #model prediction
        outputs = model(inputs)
        
        #loss
        loss = criterion(outputs,targets)
        print('loss:',loss.data.numpy()[0])
        #backpropagation
        loss.backward()
        
        # debugging
        for param in model.parameters():
            print(param.grad.data.sum()/len(param.grad.data))

        
        #gradient descent 
        optimizer.step()

    # print/store loss
    clear_output(wait=True)
    print(running_loss/b)
    losses.append(running_loss/b)
    running_loss=0

KeyboardInterrupt: 